In [2]:
from haystack.file_converter.pdf import PDFToTextConverter
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])

In [4]:
import os 
os.getcwd()

'/home/elena/Projects/neurocode/alexa-for-docs'

In [5]:
book = converter.convert(file_path="/home/elena/Downloads/9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf", meta=None)

In [23]:
type(book), len(book), book.keys(), len(book["text"])

(dict, 2, dict_keys(['text', 'meta']), 577807)

In [32]:
type(book["meta"])

NoneType

In [33]:
book["text"][100:10000]

' Learning with Keras\n\nMatthew Moocarme\nMahla Abdolahnejad\nRitesh Bhagwat\n\nThe Deep Learning with Keras Workshop\nSecond Edition\nCopyright © 2020 Packt Publishing\nAll rights reserved. No part of this book may be reproduced, stored in a retrieval system,\nor transmitted in any form or by any means, without the prior written permission of the\npublisher, except in the case of brief quotations embedded in critical articles or reviews.\nEvery effort has been made in the preparation of this book to ensure the accuracy of\nthe information presented. However, the information contained in this book is sold\nwithout warranty, either express or implied. Neither the authors, nor Packt Publishing,\nand its dealers and distributors will be held liable for any damages caused or alleged to\nbe caused directly or indirectly by this book.\nPackt Publishing has endeavored to provide trademark information about all of the\ncompanies and products mentioned in this book by the appropriate use of ca

In [12]:
from haystack.preprocessor.preprocessor import PreProcessor

processor = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="passage",
                         split_respect_sentence_boundary=False)

[nltk_data] Downloading package punkt to /home/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
doc = processor.process(book)

In [16]:
type(doc), len(doc), len(doc[0]), len(doc[1])

(list, 2, 2, 2)

In [19]:
type(doc[0]), type(doc[1]), doc[0].keys(), doc[1].keys()

(dict, dict, dict_keys(['text', 'meta']), dict_keys(['text', 'meta']))

In [25]:
len(doc[0]["text"]), len(doc[1]["text"])

(369553, 206464)

In [35]:
doc[0]["text"][100:10000]

'Learning with Keras Matthew Moocarme\nMahla Abdolahnejad\nRitesh Bhagwat The Deep Learning with Keras Workshop\nSecond Edition\nCopyright © 2020 Packt Publishing\nAll rights reserved. No part of this book may be reproduced, stored in a retrieval system,\nor transmitted in any form or by any means, without the prior written permission of the\npublisher, except in the case of brief quotations embedded in critical articles or reviews.\nEvery effort has been made in the preparation of this book to ensure the accuracy of\nthe information presented. However, the information contained in this book is sold\nwithout warranty, either express or implied. Neither the authors, nor Packt Publishing,\nand its dealers and distributors will be held liable for any damages caused or alleged to\nbe caused directly or indirectly by this book.\nPackt Publishing has endeavored to provide trademark information about all of the\ncompanies and products mentioned in this book by the appropriate use of capitals.

In [40]:
processor2 = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="sentence",
                         split_length = 260,
                         split_respect_sentence_boundary=True)
                    

[nltk_data] Downloading package punkt to /home/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
sent = processor2.process(book)

In [45]:
type(sent), len(sent), type(sent[0]), sent[0].keys(), len(sent[0]["text"]), len(sent[9]["text"])

(list, 14, dict, dict_keys(['text', 'meta']), 53981, 30844)

In [47]:
sent[2]["meta"]

{'_split_id': 2}

In [66]:
processor3 = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="word",
                         split_length = 260,
                         split_respect_sentence_boundary=True)

[nltk_data] Downloading package punkt to /home/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
word=processor3.process(book)

12/01/2020 16:57:20 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.


In [71]:
type(word), len(word), word[0].keys(), word[0]["meta"]

(list, 329, dict_keys(['text', 'meta']), {'_split_id': 0})

In [82]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()
document_store.delete_all_documents()

In [83]:
document_store.write_documents(word)

In [84]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=False,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
    

12/01/2020 17:01:11 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/01/2020 17:01:18 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [85]:
document_store.update_embeddings(retriever)

essage: Truncation error: Specified max length is too low to respect the various constraints
12/01/2020 17:01:23 - ERROR - farm.data_handler.processor -   Could not create sample(s) from this dict: 
 {'passages': [{'title': '', 'text': "Additionally, the upper bound value here is 1258, which is the index or count of\nrows (or records) in the training set:\nX_train = []\ny_train = []\nfor i in range(60, 1258):\nX_train, y_train = np.array(X_train), np.array(y_train)\n\n406 | Appendix\n6.Reshape the data to add an extra dimension to the end of X_train using NumPy's\nreshape function:\n\n7.Import the following libraries to build the RNN:\nfrom keras.models import Sequential\nfrom keras.layers import Dense, LSTM, Dropout\n\n8.Set the seed and initiate the sequential model, as follows:\nseed = 1\nnp.random.seed(seed)\nrandom.set_seed(seed)\nmodel = Sequential()\n\n9.Add an LSTM layer to the network with 50 units, set the return_sequences\nargument to True, and set the input_shape argument t

TypeError: 'NoneType' object is not iterable

In [52]:
from haystack.reader.farm import FARMReader
farm_reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

 | 360M/496M [01:10<00:49, 2.77MB/s]
Downloading:  73%|███████▎  | 360M/496M [01:10<00:46, 2.91MB/s]
Downloading:  73%|███████▎  | 361M/496M [01:10<00:44, 3.03MB/s]
Downloading:  73%|███████▎  | 361M/496M [01:10<00:42, 3.19MB/s]
Downloading:  73%|███████▎  | 361M/496M [01:10<00:40, 3.31MB/s]
Downloading:  73%|███████▎  | 362M/496M [01:10<00:37, 3.56MB/s]
Downloading:  73%|███████▎  | 362M/496M [01:10<00:34, 3.86MB/s]
Downloading:  73%|███████▎  | 363M/496M [01:11<00:34, 3.83MB/s]
Downloading:  73%|███████▎  | 363M/496M [01:11<00:31, 4.16MB/s]
Downloading:  73%|███████▎  | 364M/496M [01:11<00:30, 4.37MB/s]
Downloading:  73%|███████▎  | 364M/496M [01:11<00:27, 4.72MB/s]
Downloading:  74%|███████▎  | 365M/496M [01:11<00:28, 4.66MB/s]
Downloading:  74%|███████▎  | 365M/496M [01:11<00:26, 4.95MB/s]
Downloading:  74%|███████▎  | 366M/496M [01:11<00:26, 4.97MB/s]
Downloading:  74%|███████▍  | 367M/496M [01:11<00:26, 4.83MB/s]
Downloading:  74%|███████▍  | 367M/496M [01:11<00:24, 5.32MB/s]
Dow

In [86]:
from haystack import Finder

finder = Finder(farm_reader, retriever)

In [87]:
prediction = finder.get_answers(question="layer", top_k_retriever=1, top_k_reader=5)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.59 Batches/s]
12/01/2020 17:01:37 - INFO - haystack.finder -   Got 0 candidates from retriever
12/01/2020 17:01:37 - INFO - haystack.finder -   Retriever did not return any documents. Skipping reader ...


In [64]:
from haystack.utils import print_answers

In [81]:
print_answers(prediction, details="medium")

[]


In [88]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

12/01/2020 17:04:40 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.434s]
12/01/2020 17:04:40 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.196s]


In [89]:
document_store.write_documents(word)

12/01/2020 17:05:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.863s]


In [90]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [92]:
finder = Finder(farm_reader, retriever)

In [93]:
prediction = finder.get_answers(question="What is RNN?", top_k_retriever=10, top_k_reader=5)

12/01/2020 17:06:10 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.205s]
12/01/2020 17:06:10 - INFO - haystack.finder -   Got 10 candidates from retriever
12/01/2020 17:06:10 - INFO - haystack.finder -   Reader is looking for detailed answer in 16574 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.99s/ Batches]


In [94]:
print_answers(prediction, details="medium")

[   {   'answer': 'Recurrent Neural Networks',
        'context': 'rthermore, we will learn\n'
                   'how sequential modeling is related to Recurrent Neural '
                   'Networks (RNN).We will\n'
                   'learn about the vanishing gradient problem in d',
        'score': 16.642990112304688},
    {   'answer': 'Recurrent Neural Networks',
        'context': ' chapter, you will\n'
                   'be able to build sequential models, explain Recurrent '
                   'Neural Networks (RNNs),\n'
                   'describe the vanishing gradient problem, and implemen',
        'score': 15.48896312713623},
    {   'answer': 'Recurrent Neural Networks',
        'context': ' however, here, the data and information also flow in '
                   'cycles:\n'
                   '\n'
                   'Recurrent Neural Networks (RNNs) | 285\n'
                   '\n'
                   'Figure 9.6: An RNN\n'
                   '\n'
                   

In [95]:
print_answers(finder.get_answers(question="What is a layer?", top_k_retriever=10, top_k_reader=5), details="medium")

12/01/2020 17:07:35 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.031s]
12/01/2020 17:07:35 - INFO - haystack.finder -   Got 10 candidates from retriever
12/01/2020 17:07:35 - INFO - haystack.finder -   Reader is looking for detailed answer in 15810 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.19 Batches/s][   {   'answer': 'user-defined layer',
        'context': 'tially, what we will do is\n'
                   'replace the last layer of VGG16 with a user-defined '
                   'layer.Before we begin, ensure you have downloaded the '
                   'image datasets fr',
        'score': 10.711793899536133},
    {   'answer': 'single hidden layer',
        'context': 'will stick\n'
                   'to the simplest case, which is a neural network with a '
                   'single hidden layer.You will learn\n'
                   'how to define a model in Keras, choose the hyperp',
        'score': 7.02